In [1]:
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import pandas as pd
import re
import csv
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


### Set the Company Name

In [2]:
company_name = "中國信託"

### Navigate to the website

In [3]:
driver = webdriver.Chrome()
driver.get('https://www.fsc.gov.tw/ch/home.jsp?id=131&parentpath=0,2')

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


### Set the waiting time

In [4]:
wait = WebDriverWait(driver, 10)

### Type in the name of the company

In [5]:
company_name_input = driver.find_element("xpath", '//*[@id="keyword"]')
company_name_input.send_keys(f"{company_name}")

### Click the `Search` button

In [6]:
search_button = driver.find_element('xpath', '//*[@id="fat-search"]/div/div/div[4]/div[2]/input[1]')
search_button.click()

### Extract the whole data table

In [12]:
data_table = driver.find_elements('xpath', '//*[@id="messageform"]/div[2]/ul/li')
print(data_table[0].text.split('\n'))
print(data_table[1].text.split('\n'))
print(data_table[2].text.split('\n'))
print(data_table[0].text.split('\n')[0])
print(data_table[2].text.split('\n')[0])

['編號', '裁處書發文日期', '資料來源', '標題']
['1', '2023-08-10', '保險局', '台灣人壽保險股份有限公司公司治理運作之內部控制制度未能有效運作，且核有有......']
['2', '2023-08-10', '銀行局', '中國信託金融控股股份有限公司及相關子公司之公司治理有重大缺失，內部控制制......']
編號
2


In [42]:
# Extract the data rows from the table
data_table = driver.find_elements('xpath', '//*[@id="messageform"]/div[2]/ul/li')

# Initialize an empty list to store the data
data_list = []

# Iterate through each row and extract the required information
for row in data_table[1:]:
    data = row.text.split('\n')
    title = row.find_element(By.TAG_NAME, "a").get_attribute('title')
    entry = {
        "編號": data[0],
        "裁處書發文日期": data[1],
        "資料來源": data[2],
        "標題": title
    }
    data_list.append(entry)

# Drop the first row from data_list
# data_list = data_list[1:]

# Write the data to a CSV file
csv_file = f"{company_name}_ViolationItems.csv"
with open(csv_file, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["編號", "裁處書發文日期", "資料來源", "標題"])
    writer.writeheader()
    writer.writerows(data_list)

# Close the browser
driver.quit()

print(f"CSV file '{csv_file}' has been created with the extracted data.")


CSV file '中國信託_ViolationItems.csv' has been created with the extracted data.
